In [2]:
import os
import cv2
import numpy as np

In [46]:
#  _____ _____ _____ _____ _____ 
# |     |   __|     |     |   __|
# | | | |__   |  |  |  |  |__   |
# |_|_|_|_____|_____|_____|_____|
#  _____ _____ _____ _____ _____ 
# |   __|  _  |     |   __|   __|
# |  |  |     | | | |   __|__   |
# |_____|__|__|_|_|_|_____|_____|


# Define paths
base_folder = '../exps/videogamebench/incredible_machine/claude-3.7/'
game_screen_folder = os.path.join(base_folder, 'game_screen')
monitor_folder = os.path.join(base_folder, 'monitor') 
output_video_path = os.path.join(base_folder, 'output_video.mp4')
frame_rate = 5

# Get sorted list of image filenames
game_screen_files = sorted([
    f for f in os.listdir(game_screen_folder)
    if f.startswith('game_screen_step_') and f.endswith('.jpg')
], key=lambda x: int(x.split('_')[3].split('.')[0]))
monitor_files = sorted([
    f for f in os.listdir(monitor_folder)
    if f.startswith('game_screen_step_') and f.endswith('.jpg')
], key=lambda x: int(x.split('_')[3].split('.')[0]))

# Ensure equal number of frames
num_frames = min(len(game_screen_files), len(monitor_files))
game_screen_files = game_screen_files[:num_frames]
monitor_files = monitor_files[:num_frames]

print(len(game_screen_files), len(monitor_files))

# Load first images to determine sizes
img_game_screen = cv2.imread(os.path.join(game_screen_folder, game_screen_files[0]))
img_monitor = cv2.imread(os.path.join(monitor_folder, monitor_files[0]))

# Crop game screen to 640x400 
cropped_game_screen = img_game_screen[0:400, 0:640]

# Resize game screen to match monitor height
monitor_height = img_monitor.shape[0]
scale_ratio = monitor_height / cropped_game_screen.shape[0]
new_game_screen_width = int(cropped_game_screen.shape[1] * scale_ratio)

# Determine the size of the combined frame
frame_width = new_game_screen_width + img_monitor.shape[1]
frame_height = monitor_height

# Initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (frame_width, frame_height))

# Process and write frames
for gs_file, mon_file in zip(game_screen_files, monitor_files):
    img_game = cv2.imread(os.path.join(game_screen_folder, gs_file))
    img_mon = cv2.imread(os.path.join(monitor_folder, mon_file))

    # Crop game screen to 640x400 then chop off the left 50 pixels
    img_game_cropped = img_game[0:400, 0:640]

    # Resize game screen image
    img_game_resized = cv2.resize(img_game_cropped, (new_game_screen_width, monitor_height))

    # Concatenate images side by side
    combined_img = np.hstack((img_game_resized, img_mon))

    # Write to video
    out.write(combined_img)

# Release the video writer
out.release()
print(f"Video saved to {output_video_path}")


200 200
Video saved to ../exps/videogamebench/incredible_machine/claude-3.7/output_video.mp4


In [15]:
import os
import cv2
import numpy as np

#  _____  ____   ___    _____                         
# / ____||  _ \ / _ \  / ____|                        
#| |  __ | |_) | |_| || |  __   __ _ _ __ ___   ___  
#| | |_ ||  _ <|  _  || | |_ | / _` | '_ ` _ \ / _ \ 
#| |__| || |_) | | | || |__| || (_| | | | | | |  __/ 
# \_____||____/|_| |_| \_____| \__,_|_| |_| |_|\___| 

# Define paths
base_folder = '../exps/videogamebench/zelda/gemini-2.5-pro/'
emulator_image_path = '../media/img/emulator.png'
game_screen_folder = os.path.join(base_folder, 'game_screen')
monitor_folder = os.path.join(game_screen_folder, 'monitor')
output_video_path = os.path.join(base_folder, 'output_video.mp4')
frame_rate = 20

# Define emulator screen position (x, y, width, height)
emulator_screen_rect = (60, 60, 140, 140)  # Example values

# Get sorted list of image filenames by frame number
game_screen_files = sorted([
    f for f in os.listdir(game_screen_folder)
    if f.startswith('screenshot_') and f.endswith('.png')
], key=lambda x: int(x.split('_')[1].split('.')[0]))
monitor_files = sorted([
    f for f in os.listdir(monitor_folder)
    if f.startswith('screenshot_') and f.endswith('.jpg')
], key=lambda x: int(x.split('_')[1].split('.')[0]))

# Ensure equal number of frames
print(len(game_screen_files), len(monitor_files))
num_frames = min(len(game_screen_files), len(monitor_files))
game_screen_files = game_screen_files[0:num_frames]
monitor_files = monitor_files[0:num_frames]

# Load first images to determine sizes
img_game_screen = cv2.imread(os.path.join(game_screen_folder, game_screen_files[0]))
img_monitor = cv2.imread(os.path.join(monitor_folder, monitor_files[0]))
img_emulator = cv2.imread(emulator_image_path)

# Emulator screen dimensions and frame height
em_x, em_y, em_w, em_h = emulator_screen_rect
frame_height = max(img_emulator.shape[0], img_monitor.shape[0])

# Resize emulator image to match frame height
scale_ratio = frame_height / img_emulator.shape[0]
img_emulator_resized = cv2.resize(img_emulator, None, fx=scale_ratio, fy=scale_ratio, interpolation=cv2.INTER_LINEAR)

# Update emulator screen rect coordinates after resizing
em_x = int(em_x * scale_ratio)
em_y = int(em_y * scale_ratio)
em_w = int(em_w * scale_ratio)
em_h = int(em_h * scale_ratio)

# Determine the size of the combined frame
frame_width = img_emulator_resized.shape[1] + img_monitor.shape[1]

# Initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (frame_width, frame_height))

# Define the game title
game_title = "Gemini 2.5 Pro"

# Font settings
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1.0
font_thickness = 2
text_color = (0, 255, 0)

# Process and write frames
for gs_file, mon_file in zip(game_screen_files, monitor_files):
    img_game = cv2.imread(os.path.join(game_screen_folder, gs_file))
    img_mon = cv2.imread(os.path.join(monitor_folder, mon_file))
    base_emulator = img_emulator_resized.copy()

    # ----- Add title text (no background) -----
    (text_width, text_height), _ = cv2.getTextSize(game_title, font, font_scale, font_thickness)
    text_x = (base_emulator.shape[1] - text_width) // 2
    text_y = text_height + 30
    cv2.putText(base_emulator, game_title, (text_x, text_y), font, font_scale, text_color, font_thickness, cv2.LINE_AA)
    # ------------------------------------------

    # Resize game screen image to fit emulator screen
    img_game_resized = cv2.resize(img_game, (em_w, em_h))

    # Overlay resized game screen onto emulator
    base_emulator[em_y:em_y+em_h, em_x:em_x+em_w] = img_game_resized

    # Create combined frame by concatenating emulator with monitor image
    padded_monitor = np.zeros((frame_height, img_monitor.shape[1], 3), dtype=np.uint8)
    padded_monitor[:img_monitor.shape[0], :] = img_mon

    combined_img = np.hstack((base_emulator, padded_monitor))

    # Write to video
    out.write(combined_img)

# Release the video writer
out.release()
print(f"Video saved to {output_video_path}")


2930 2930
Video saved to ../exps/videogamebench/zelda/gemini-2.5-pro/output_video.mp4
